In [17]:
pip install keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [19]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [20]:
fashion_mnist=keras.datasets.fashion_mnist

In [21]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [22]:
train_images=train_images/255.0

In [23]:
test_images=test_images/255.0

In [24]:
train_images[0].shape

(28, 28)

In [25]:
train_images=train_images.reshape(len(train_images),28,28,1)

In [26]:
test_images=test_images.reshape(len(test_images),28,28,1)

In [27]:
def build_model(hp):
    model=keras.Sequential([keras.layers.Conv2D(
    filters=hp.Int('conv_1_filter',min_value=32,max_value=128,step=16),
    kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
    activation='relu',
    input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
    filters=hp.Int('conv_2_filter',min_value=32,max_value=64,step=16),
    kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
    activation='relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(10,activation='softmax') #output layer
        ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
                            
    return model

In [37]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [38]:

tuner_search=RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name="Mnist Fashion")
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Reloading Oracle from existing project output\Mnist Fashion\oracle.json
INFO:tensorflow:Reloading Tuner from output\Mnist Fashion\tuner0.json
INFO:tensorflow:Oracle triggered exit


In [39]:
model=tuner_search.get_best_models(num_models=1)[0]

In [40]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        134448    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                232330    
Total params: 367,898
Trainable params: 367,898
Non-trainable params: 0
_________________________________________________________________
